In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Optional for date parsing
from datetime import datetime


In [3]:
# import requests

# url = "https://your-cloud-storage-link/train.csv"
# r = requests.get(url)
# with open("train.csv", "wb") as f:
#     f.write(r.content)


In [4]:
df_sales = pd.read_csv('../data/train.csv', parse_dates=['date'])
df_stores = pd.read_csv('../data/stores.csv')
df_txns = pd.read_csv('../data/transactions.csv', parse_dates=['date'])

In [7]:
df_sales.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [8]:
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [9]:
df_txns.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [10]:
# Merge sales with store metadata
df = pd.merge(df_sales, df_stores, on='store_nbr', how='left')

# Merge with transactions
df = pd.merge(df, df_txns, on=['date', 'store_nbr'], how='left')

# Sort and inspect
df.sort_values(by=['store_nbr', 'date'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()


,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN


In [12]:
# Cleanup

# Fill missing transaction values (if any)
df['transactions'] = df['transactions'].fillna(0)

# Create year-month column
df['year_month'] = df['date'].dt.to_period('M').astype(str)

# Preview structure
df[['date', 'year_month', 'store_nbr', 'family', 'sales', 'transactions', 'city', 'type']].head()


,date,year_month,store_nbr,family,sales,transactions,city,type
0,2013-01-01,2013-01,1,AUTOMOTIVE,0.0,0.0,Quito,D
1,2013-01-01,2013-01,1,BABY CARE,0.0,0.0,Quito,D
2,2013-01-01,2013-01,1,BEAUTY,0.0,0.0,Quito,D
3,2013-01-01,2013-01,1,BEVERAGES,0.0,0.0,Quito,D
4,2013-01-01,2013-01,1,BOOKS,0.0,0.0,Quito,D


In [13]:
# Export Cleaned File for SAC
# Save relevant columns to a new CSV
df_export = df[['date', 'store_nbr', 'family', 'sales', 'transactions', 'city', 'type', 'cluster']]
df_export.to_csv('../data/sales_clean.csv', index=False)
